**Stock Market Return Prediction(Time Series Problem)**

In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import pandas_ta as ta
from keras.layers import Conv1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from sklearn.metrics import mean_squared_error

## Data

In [2]:
df = pd.read_csv('sp500.csv', delimiter = ' ', header=None, skiprows=1, names=["Index ", "GSPC.Open", "GSPC.High", "GSPC.Low", "GSPC.Close", "GSPC.Volume", "GSPC.Adjusted"],engine='python')
df.head()

,Index,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted
0,1970-01-02,92.059998,93.540001,91.790001,93.000000,8050000.0,93.000000
1,1970-01-05,93.000000,94.250000,92.529999,93.459999,11490000.0,93.459999
2,1970-01-06,93.459999,93.809998,92.129997,92.820000,11460000.0,92.820000
3,1970-01-07,92.820000,93.379997,91.930000,92.629997,10010000.0,92.629997
4,1970-01-08,92.629997,93.470001,91.989998,92.680000,10670000.0,92.680000


In [3]:
#Number of missing values in the dataset
df.isna().sum()

Index            0
GSPC.Open        0
GSPC.High        0
GSPC.Low         0
GSPC.Close       0
GSPC.Volume      0
GSPC.Adjusted    0
dtype: int64

## Defining Prediction Task

This project aims to have good forecasts of the future price of the S&P500 index so that profitable orders can be placed on time. Below we describe a variable, calculated with the quotes data, that can be seen as an indicator (a value) of the tendency in the next k days. The value of this indicator should be related to the confidence we have that the target
margin p will be attainable in the next k days.

In [4]:
df['Daily_average_price'] = (df['GSPC.High']+df['GSPC.Low']+df['GSPC.Close'])/3
df.head()

,Index,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted,Daily_average_price
0,1970-01-02,92.059998,93.540001,91.790001,93.000000,8050000.0,93.000000,92.776667
1,1970-01-05,93.000000,94.250000,92.529999,93.459999,11490000.0,93.459999,93.413333
2,1970-01-06,93.459999,93.809998,92.129997,92.820000,11460000.0,92.820000,92.919998
3,1970-01-07,92.820000,93.379997,91.930000,92.629997,10010000.0,92.629997,92.646665
4,1970-01-08,92.629997,93.470001,91.989998,92.680000,10670000.0,92.680000,92.713333


In [5]:
def T_index(n_days=120, margin=0.025):
    T_list = []
    for i in df.index:
        T = 0
        Ci = df['GSPC.Close'].iloc[i]
        for j in range (i + 1, n_days + 1):
            variation = (df['Daily_average_price'].iloc[j] - Ci) / Ci
            if (variation > margin) or (variation < -margin) :
                T = T + variation
        T_list.append(T)
    return(T_list)

In [6]:
df['T_index'] = T_index()
df['T_index'].value_counts()

 0.000000    11508
-1.132061        2
-6.501096        2
-0.220323        2
-3.395769        1
             ...  
-4.646347        1
 0.828822        1
-5.059725        1
-4.910865        1
-2.369302        1
Name: T_index, Length: 112, dtype: int64

The general idea of the variable T is to signal k-days periods that have
several days with average daily prices clearly above the target variation. High
positive values of T mean that there are several average daily prices that are
p% higher than today’s close. Such situations are good indications of potential
opportunities to issue a buy order, as we have good expectations that the prices
will rise. On the other hand, highly negative values of T suggest sell actions,
given the prices will probably decline. Values around zero can be caused by
periods with “flat” prices or by conflicting positive and negative variations
that cancel each other.

## Defining the Predictors

We have defined an indicator (T) that summarizes the behavior of the price time series in the next k days. We are approximating the future behavior (f), by our indicator T. We now have to decide on how we will describe the recent prices pattern (p in the description above). Instead of using again a single indicator to describe these recent dynamics, we will use several thechnical indicators, trying to capture different properties of the price time series to facilitate the forecasting task. The technical indicators which were used in this task are the Average True Value, Directional Movement Index, Parabolic SAR, Moving Average, Arms Ease of Movement, Volatility OHC, Percentage Change and MACD oscillator.

In [7]:
# macd indicator
macd = df.ta.macd(close='GSPC.Close', append=False)
df['MACD'] = macd['MACD_12_26_9']

# atr indicator
atr = df.ta.atr(high='GSPC.High', low='GSPC.LOW', close='GSPC.Close', length=10, append=False)
df['ATR'] = atr

# adx indicator
adx = df.ta.adx(high='GSPC.High', low='GSPC.LOW', close='GSPC.Close', length=10, append=False)
df['ADX'] = adx['ADX_10']

# psar indicator
psar = df.ta.psar(high='GSPC.High', low='GSPC.LOW', close='GSPC.Close', append=False)
df['PSAR'] = psar['PSARl_0.02_0.2'].fillna(psar['PSARs_0.02_0.2'])

# evm indicator
def EVM(data, ndays): 
 dm = ((data['GSPC.High'] + data['GSPC.Low'])/2) - ((data['GSPC.High'].shift(1) + data['GSPC.Low'].shift(1))/2)
 br = (data['GSPC.Volume'] / 100000000) / ((data['GSPC.High'] - data['GSPC.Low']))
 EVM = dm / br 
 EVM_MA = pd.Series(EVM.rolling(ndays).mean(), name = 'EVM')
 return EVM_MA

emv = EVM(pd.read_csv('sp500.csv', delimiter = ' ', header=None, skiprows=1, names=["Index ", "GSPC.Open", "GSPC.High", "GSPC.Low", "GSPC.Close", "GSPC.Volume", "GSPC.Adjusted"],engine='python'), 10)
df['EMV'] = emv

# runMean indicator
def runMean (column_series, window_size):
    windows = column_series.rolling(window_size)
    # Get the window of series
    # of observations of specified window size
    windows = column_series.rolling(window_size)
  
    # Create a series of moving
    # averages of each window
    moving_averages = windows.mean()
  
    # Convert pandas series back to list
    moving_averages_list = moving_averages.tolist()
  
    # Remove null entries from the list
    final_list = moving_averages_list[window_size - 1:]
  
    return(final_list)
    
runMean = runMean(df['GSPC.Close'], 10)
runMean.extend([0, 0, 0, 0, 0, 0, 0, 0, 0])
df['runMean'] = runMean

# Volat indicator
volat = np.sqrt(1 / 10 * pd.DataFrame.rolling(0.5 * np.log(df.loc[:, 'GSPC.High'] / df.loc[:, 'GSPC.Low']) ** 2 - (2 * np.log(2) - 1) * np.log(df.loc[:, 'GSPC.Close'] / df.loc[:, 'GSPC.Open']) ** 2, window=10).sum())
df['Volat'] = volat

# Delt indicator
df['DELT'] = df['GSPC.Close'].pct_change(periods=10)

In [8]:
df.head()

,Index,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted,Daily_average_price,T_index,MACD,ATR,ADX,PSAR,EMV,runMean,Volat,DELT
0,1970-01-02,92.059998,93.540001,91.790001,93.000000,8050000.0,93.000000,92.776667,-11.200394,NaN,NaN,NaN,NaN,NaN,92.393999,NaN,NaN
1,1970-01-05,93.000000,94.250000,92.529999,93.459999,11490000.0,93.459999,93.413333,-11.686674,NaN,NaN,NaN,94.2500,NaN,92.185999,NaN,NaN
2,1970-01-06,93.459999,93.809998,92.129997,92.820000,11460000.0,92.820000,92.919998,-11.008798,NaN,NaN,NaN,94.2500,NaN,91.805000,NaN,NaN
3,1970-01-07,92.820000,93.379997,91.930000,92.629997,10010000.0,92.629997,92.646665,-10.805747,NaN,NaN,NaN,94.2500,NaN,91.506000,NaN,NaN
4,1970-01-08,92.629997,93.470001,91.989998,92.680000,10670000.0,92.680000,92.713333,-10.859265,NaN,NaN,NaN,94.1108,NaN,91.238000,NaN,NaN


In [9]:
data = df.copy(deep=True)

In [10]:
data.dropna(inplace=True)
data.reset_index(inplace=True, drop=True)
data.head()

,Index,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted,Daily_average_price,T_index,MACD,ATR,ADX,PSAR,EMV,runMean,Volat,DELT
0,1970-02-06,85.900002,86.879997,85.230003,86.330002,10150000.0,86.330002,86.146667,-4.028883,-2.398583,1.867364,65.075354,84.731522,-5.850815,86.769002,0.015200,-0.034016
1,1970-02-09,86.330002,87.849998,86.160004,87.010002,10830000.0,87.010002,87.006668,-4.689271,-2.231210,1.848402,59.406594,84.848662,-2.354397,86.939001,0.015286,-0.013156
2,1970-02-10,87.010002,87.400002,85.580002,86.099998,10110000.0,86.099998,86.360001,-3.785442,-2.147242,1.845386,55.549605,85.028742,-2.221681,87.037001,0.015362,-0.017348
3,1970-02-11,86.099998,87.379997,85.300003,86.940002,12260000.0,86.940002,86.540001,-4.646347,-1.989977,1.870143,52.665623,85.198017,-1.808237,87.362001,0.015608,0.001728
4,1970-02-12,86.940002,87.540001,85.930000,86.730003,10010000.0,86.730003,86.733335,-4.517162,-1.860838,1.842843,49.659310,85.300003,1.005585,87.558001,0.015427,0.012137


## Build Model

In [11]:
X = data[['MACD', 'ATR', 'ADX', 'EMV', 'PSAR', 'Volat', 'DELT', 'runMean']]
y = data['T_index']

# Split to Train and Test with Sliding Window

In [20]:
from sklearn.preprocessing import MinMaxScaler
import math
from math import *

sliding_window = 50

#splitting data into training (75%) and test(25%) sets
training_dataset_length = math.ceil(len(X) * .75)

sc = MinMaxScaler(feature_range=(0, 1))

train_data = X.iloc[0:training_dataset_length, :]
SC = sc.fit(train_data)
train_data = SC.transform(train_data)
test_data = X.iloc[training_dataset_length-sliding_window:, :]
test_data = SC.transform(test_data)

In [21]:
# x_train will contain values of sliding windows
# y_train will contain values of every sliding window+1 values which we want to predict
x_train = []
y_train = []

for i in range(sliding_window, training_dataset_length):
    x_train.append(train_data[i-sliding_window:i, :])
    y_train.append(y[i])

    
# y_test will contain values of every sliding window+1 values which we want to predict
y_test = y[training_dataset_length:]

# x_test will contain values of sliding windows
x_test = []
for i in range(sliding_window, len(test_data)):
    x_test.append(test_data[i-sliding_window:i, :])

In [22]:
# Convert to numpy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)

In [23]:
x_train.shape

(8648, 50, 8)

## Modeling Tools

### MLP

In [24]:
# flatten input
n_input = x_train.shape[1] * x_train.shape[2]
x_train_2D = x_train.reshape((x_train.shape[0], n_input))
n_input = x_test.shape[1] * x_test.shape[2]
x_test_2D = x_test.reshape((x_test.shape[0], n_input))

In [25]:
# Initialising the MLP
from keras.models import Sequential
from keras.layers import Dense

MLPmodel = Sequential()
MLPmodel.add(Dense(100, activation='relu', input_dim=x_train_2D.shape[1]))
MLPmodel.add(Dense(1))
MLPmodel.compile(optimizer='adam', loss='mse')

# fit model
MLPmodel.fit(x_train_2D, y_train, epochs=100, verbose=1)

# check predicted values
predictions = MLPmodel.predict(x_test_2D)

Epoch 1/100
271/271 [==============================] - 2s 2ms/step - loss: 0.0191
Epoch 2/100
271/271 [==============================] - 0s 2ms/step - loss: 0.0137
Epoch 3/100
271/271 [==============================] - 0s 2ms/step - loss: 0.0130
Epoch 4/100
271/271 [==============================] - 0s 2ms/step - loss: 0.0132
Epoch 5/100
271/271 [==============================] - 0s 2ms/step - loss: 0.0126
Epoch 6/100
271/271 [==============================] - 1s 2ms/step - loss: 0.0128
Epoch 7/100
271/271 [==============================] - 0s 2ms/step - loss: 0.0127
Epoch 8/100
271/271 [==============================] - 1s 2ms/step - loss: 0.0127
Epoch 9/100
271/271 [==============================] - 1s 2ms/step - loss: 0.0128
Epoch 10/100
271/271 [==============================] - 1s 2ms/step - loss: 0.0122
Epoch 11/100
271/271 [==============================] - 1s 2ms/step - loss: 0.0126
Epoch 12/100
271/271 [==============================] - 1s 2ms/step - loss: 0.0119
Epoch 13/100


In [19]:
error = np.sqrt(mean_squared_error(y_test, predictions, multioutput='uniform_average'))
print('MLP_RMSE:', error)

MLP_RMSE: 0.011273219977976747


### LSTM

In [20]:
#import math
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dropout

# Initialising the LSTM
model = Sequential()

model.add(LSTM(units=200, return_sequences=True, activation='relu',input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model.add(LSTM(units=200,activation='relu', return_sequences=True))

# Adding a third LSTM layer and Dropout layer
model.add(LSTM(units=200, return_sequences=True,activation='relu'))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and and Dropout layer
model.add(LSTM(units=200))
model.add(Dropout(0.2))

# Adding the output layer
# For Full connection layer we use dense
# As the output is 1D so we use unit=1
model.add(Dense(units=1))

# compile and fit the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=5, batch_size=500, verbose=True)

Epoch 1/5
18/18 [==============================] - 108s 5s/step - loss: 0.0122
Epoch 2/5
18/18 [==============================] - 90s 5s/step - loss: 0.0118
Epoch 3/5
18/18 [==============================] - 97s 5s/step - loss: 0.0118
Epoch 4/5
18/18 [==============================] - 105s 6s/step - loss: 0.0118
Epoch 5/5
18/18 [==============================] - 97s 5s/step - loss: 0.0118


In [21]:
error = np.sqrt(mean_squared_error(y_test, predictions, multioutput='uniform_average'))
print('LSTM_RMSE:', error)

LSTM_RMSE: 0.02871761071832099


### CNN

In [23]:
# define model
modelCNN = Sequential()
modelCNN.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
modelCNN.add(MaxPooling1D(pool_size=2))
modelCNN.add(Flatten())
modelCNN.add(Dense(50, activation='relu'))
modelCNN.add(Dense(1))
modelCNN.compile(optimizer='adam', loss='mse')

# fit model
modelCNN.fit(x_train, y_train, epochs=50, verbose=1)

Epoch 1/50
271/271 [==============================] - 4s 5ms/step - loss: 0.0142
Epoch 2/50
271/271 [==============================] - 1s 4ms/step - loss: 0.0123
Epoch 3/50
271/271 [==============================] - 1s 5ms/step - loss: 0.0118
Epoch 4/50
271/271 [==============================] - 1s 5ms/step - loss: 0.0116
Epoch 5/50
271/271 [==============================] - 1s 5ms/step - loss: 0.0114
Epoch 6/50
271/271 [==============================] - 1s 5ms/step - loss: 0.0115
Epoch 7/50
271/271 [==============================] - 1s 5ms/step - loss: 0.0114
Epoch 8/50
271/271 [==============================] - 1s 5ms/step - loss: 0.0113
Epoch 9/50
271/271 [==============================] - 1s 4ms/step - loss: 0.0111
Epoch 10/50
271/271 [==============================] - 1s 4ms/step - loss: 0.0110
Epoch 11/50
271/271 [==============================] - 1s 4ms/step - loss: 0.0108
Epoch 12/50
271/271 [==============================] - 1s 5ms/step - loss: 0.0105
Epoch 13/50
271/271 [====

In [24]:
predictions = modelCNN.predict(x_test)

error = np.sqrt(mean_squared_error(y_test, predictions, multioutput='uniform_average'))
print('CNN_RMSE:', error)

CNN_RMSE: 0.015608098290853323


### SVM

In [56]:
import tslearn
from tslearn import svm 

svr = svm.TimeSeriesSVR(kernel="poly", verbose=1, max_iter=10, n_jobs=1)

In [57]:
help(svm.TimeSeriesSVR)

Help on class TimeSeriesSVR in module tslearn.svm.svm:

class TimeSeriesSVR(TimeSeriesSVMMixin, sklearn.base.RegressorMixin, tslearn.bases.bases.TimeSeriesBaseEstimator)
 |  TimeSeriesSVR(C=1.0, kernel='gak', degree=3, gamma='auto', coef0=0.0, tol=0.001, epsilon=0.1, shrinking=True, cache_size=200, n_jobs=None, verbose=0, max_iter=-1)
 |  
 |  Time-series specific Support Vector Regressor.
 |  
 |  Parameters
 |  ----------
 |  C : float, optional (default=1.0)
 |      Penalty parameter C of the error term.
 |  
 |  kernel : string, optional (default='gak')
 |       Specifies the kernel type to be used in the algorithm.
 |       It must be one of 'gak' or a kernel accepted by ``sklearn.svm.SVC``.
 |       If none is given, 'gak' will be used. If a callable is given it is
 |       used to pre-compute the kernel matrix from data matrices; that matrix
 |       should be an array of shape ``(n_samples, n_samples)``.
 |  
 |  degree : int, optional (default=3)
 |      Degree of the polynomi

In [58]:
svr.fit(x_train, y_train)
predictions = svr.predict(x_test)

[LibSVM]

C:\Users\vasia\anaconda3\envs\myenv\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [61]:
error = np.sqrt(mean_squared_error(y_test, predictions, multioutput='uniform_average'))
print('SVR_RMSE:', error)

SVR_RMSE: 4.063749122652865


### SPLINES

In [33]:
help(Earth)

Help on class Earth in module pyearth.earth:

class Earth(_Earth, sklearn.base.RegressorMixin, sklearn.base.TransformerMixin, sklearn.base.BaseEstimator)
 |  Earth(max_terms=None, max_degree=None, allow_missing=False, penalty=None, endspan_alpha=None, endspan=None, minspan_alpha=None, minspan=None, thresh=None, zero_tol=None, min_search_points=None, check_every=None, allow_linear=None, use_fast=None, fast_K=None, fast_h=None, smooth=None, enable_pruning=True, feature_importance_type=None, verbose=0)
 |  
 |  Multivariate Adaptive Regression Splines
 |  
 |  A flexible regression method that automatically searches for interactions
 |  and non-linear relationships.  Earth models can be thought of as
 |  linear models in a higher dimensional basis space
 |  (specifically, a multivariate truncated power spline basis).
 |  Each term in an Earth model is a product of so called "hinge functions".
 |  A hinge function is a function that's equal to its argument where that
 |  argument is greate

In [34]:
import pyearth
from pyearth import Earth

model = Earth(max_terms=20, verbose=True)

In [42]:
model.fit(x_train_2D, y_train)

Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.011780  1      0.012  0.000  0.000  
1     0       399  2240  0.011636  3      0.012  0.012  0.011  
2     0       348  1064  0.011010  5      0.011  0.065  0.063  
3     0       391  336   0.010770  7      0.011  0.086  0.083  
4     0       399  1210  0.010597  9      0.011  0.100  0.096  
5     0       399  1376  0.010066  11     0.010  0.145  0.141  
6     0       391  17    0.009518  13     0.010  0.192  0.186  
7     0       348  27    0.009038  15     0.009  0.233  0.226  
8     0       391  1377  0.008397  17     0.008  0.287  0.281  
9     0       204  1925  0.008202  19     0.008  0.304  0.296  
10    0       7    1308  0.008018  21     0.008  0.319  0.311  
---------------------------------------------------------------
Stopping Conditio

C:\Users\vasia\anaconda3\envs\myenv\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()


--------------------------------------------
iter  bf  terms  mse   gcv    rsq    grsq   
--------------------------------------------
0     -   21     0.01  0.009  0.284  0.275  
1     1   20     0.01  0.008  0.319  0.312  
2     10  19     0.01  0.008  0.319  0.312  
3     15  18     0.01  0.008  0.319  0.313  
4     14  17     0.01  0.008  0.319  0.313  
5     5   16     0.01  0.008  0.319  0.313  
6     11  15     0.01  0.008  0.319  0.314  
7     17  14     0.01  0.008  0.319  0.314  
8     19  13     0.01  0.008  0.319  0.315  
9     6   12     0.01  0.008  0.319  0.315  
10    2   11     0.01  0.008  0.318  0.314  
11    4   10     0.01  0.008  0.302  0.298  
12    20  9      0.01  0.008  0.284  0.281  
13    18  8      0.01  0.009  0.257  0.254  
14    8   7      0.01  0.009  0.237  0.235  
15    3   6      0.01  0.010  0.149  0.146  
16    13  5      0.01  0.010  0.148  0.147  
17    12  4      0.01  0.011  0.066  0.064  
18    16  3      0.01  0.012  0.019  0.018  
19    9   

C:\Users\vasia\anaconda3\envs\myenv\lib\site-packages\pyearth\earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


Earth(max_terms=20, verbose=True)

In [46]:
predictions = model.predict(x_test_2D)

error = np.sqrt(mean_squared_error(y_test, predictions, multioutput='uniform_average'))
print('SPLINES_RMSE:', error)

SPLINES_RMSE: 0.5093975694449377


## Trading Strategies

Our trading system can now open two types of positions: long and short. The first type is the long position.Long positions are opened by buying a commodity at time t and price p, and selling it at a later time t + x. The second type is the short position, where the trader sells the security at time t with price p with the obligation of buying it in the future.


Our trading system can now open two types of positions: long and short. The first type is the long position.Long positions are opened by buying a commodity at time t and price p, and selling it at a later time t + x. The second type is the short position, where the trader sells the security at time t with price p with the obligation of buying it in the future.


The mechanics of the first trading strategy we are going to use are the
following. First, all decisions will be taken at the end of the day, that is, after
knowing all daily quotes of the current session. Suppose that at the end of
day t, our models provide evidence that the prices are going down, that is,
predicting a low value of T or a sell signal. If we already have a position
opened, the indication of the model will be ignored. If we currently do not
hold any opened position, we will open a short position by issuing a sell order.
When this order is carried out by the market at a price pr sometime in the
future, we will immediately post two other orders. The first is a buy limit
order with a limit price of pr − p%, where p% is a target profit margin. This
type of order is carried out only if the market price reaches the target limit
price or below. This order expresses what our target profit is for the short
position just opened. We will wait 10 days for this target to be reached. If the
order is not carried out by this deadline, we will buy at the closing price of
the 10th day. The second order is a buy stop order with a price limit pr + l%.
This order is placed with the goal of limiting our eventual losses with this
position. The order will be executed if the market reaches the price pr + l%,
thus limiting our possible losses to l%.

## Summary and Decision Making

In our approach to this problem we will assume that the correct trading action at time t is related to what our expectations are concerning the evolution of prices in the next k days. Moreover, we will describe this future evolution of the prices by our indicator T. The correct trading signal at time t will be “buy” if the T score is higher than a certain threshold, and will be “sell” if the score is below another threshold. In all other cases, the correct signal will be do nothing (i.e., “hold”).


\begin{equation}
  signal =
    \begin{cases}
      sell & \text{if if T < −0.1}\\
      hold & \text{if − 0.1 ≤ T ≤ 0.1}\\
      buy & \text{if T > 0.1}
    \end{cases}       
\end{equation}

# Split to Train and Test with sklearn

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 500, shuffle=False)

## MLP

In [15]:
# Initialising the MLP
from keras.models import Sequential
from keras.layers import Dense

MLPmodel = Sequential()
MLPmodel.add(Dense(100, activation='relu', input_dim=x_train.shape[1]))
MLPmodel.add(Dense(1))
MLPmodel.compile(optimizer='adam', loss='mse')

# fit model
MLPmodel.fit(x_train, y_train, epochs=100, verbose=1)

# check predicted values
predictions = MLPmodel.predict(x_test)

Epoch 1/100
254/254 [==============================] - 3s 1ms/step - loss: 53.1984
Epoch 2/100
254/254 [==============================] - 0s 1ms/step - loss: 1.3136
Epoch 3/100
254/254 [==============================] - 0s 1ms/step - loss: 0.5693
Epoch 4/100
254/254 [==============================] - 0s 1ms/step - loss: 0.3007
Epoch 5/100
254/254 [==============================] - 0s 1ms/step - loss: 0.2622
Epoch 6/100
254/254 [==============================] - 0s 1ms/step - loss: 0.2468
Epoch 7/100
254/254 [==============================] - 0s 1ms/step - loss: 0.2244
Epoch 8/100
254/254 [==============================] - 0s 1ms/step - loss: 0.2230
Epoch 9/100
254/254 [==============================] - 0s 1ms/step - loss: 0.2268
Epoch 10/100
254/254 [==============================] - 0s 1ms/step - loss: 0.2273
Epoch 11/100
254/254 [==============================] - 0s 1ms/step - loss: 0.2185
Epoch 12/100
254/254 [==============================] - 1s 2ms/step - loss: 0.2389
Epoch 13/100

In [16]:
error = np.sqrt(mean_squared_error(y_test, predictions, multioutput='uniform_average'))
print('MLP_RMSE:', error)

MLP_RMSE: 2.9104190687872618


## LSTM

In [22]:
#import math
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dropout

# Initialising the LSTM
model = Sequential()

model.add(LSTM(units=200, return_sequences=True, activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model.add(LSTM(units=200,activation='relu', return_sequences=True))

# Adding a third LSTM layer and Dropout layer
model.add(LSTM(units=200, return_sequences=True,activation='relu'))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and and Dropout layer
model.add(LSTM(units=200))
model.add(Dropout(0.2))

# Adding the output layer
# For Full connection layer we use dense
# As the output is 1D so we use unit=1
model.add(Dense(units=1))

# compile and fit the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=5, batch_size=500, verbose=True)

Epoch 1/5
17/17 [==============================] - 18s 741ms/step - loss: 0.2999
Epoch 2/5
17/17 [==============================] - 12s 709ms/step - loss: 0.2084
Epoch 3/5
17/17 [==============================] - 12s 711ms/step - loss: 0.2057
Epoch 4/5
17/17 [==============================] - 13s 743ms/step - loss: 0.2045
Epoch 5/5
17/17 [==============================] - 12s 728ms/step - loss: 0.2033


In [23]:
error = np.sqrt(mean_squared_error(y_test, predictions, multioutput='uniform_average'))
print('LSTM_RMSE:', error)

LSTM_RMSE: 2.9104190687872618


## CNN

In [24]:
# define model
modelCNN = Sequential()
modelCNN.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(x_train.shape[1], 1)))
modelCNN.add(MaxPooling1D(pool_size=2))
modelCNN.add(Flatten())
modelCNN.add(Dense(50, activation='relu'))
modelCNN.add(Dense(1))
modelCNN.compile(optimizer='adam', loss='mse')

# fit model
modelCNN.fit(x_train, y_train, epochs=50, verbose=1)

Epoch 1/50
254/254 [==============================] - 1s 2ms/step - loss: 3.1168
Epoch 2/50
254/254 [==============================] - 0s 2ms/step - loss: 0.2189
Epoch 3/50
254/254 [==============================] - 0s 2ms/step - loss: 0.2780
Epoch 4/50
254/254 [==============================] - 0s 2ms/step - loss: 0.2482
Epoch 5/50
254/254 [==============================] - 0s 2ms/step - loss: 1.3720
Epoch 6/50
254/254 [==============================] - 0s 2ms/step - loss: 0.5303
Epoch 7/50
254/254 [==============================] - 0s 2ms/step - loss: 1.8879
Epoch 8/50
254/254 [==============================] - 0s 2ms/step - loss: 3.0550
Epoch 9/50
254/254 [==============================] - 0s 2ms/step - loss: 0.2161
Epoch 10/50
254/254 [==============================] - 0s 2ms/step - loss: 0.2114
Epoch 11/50
254/254 [==============================] - 0s 2ms/step - loss: 0.2105
Epoch 12/50
254/254 [==============================] - 0s 2ms/step - loss: 0.2506
Epoch 13/50
254/254 [====

In [25]:
predictions = modelCNN.predict(x_test)

error = np.sqrt(mean_squared_error(y_test, predictions, multioutput='uniform_average'))
print('CNN_RMSE:', error)

CNN_RMSE: 0.19451442039293135


## SVM

In [31]:
from sklearn import svm

sv = svm.SVR()
sv.fit(x_train, y_train)
y_pred = sv.predict(x_test)

error = np.sqrt(mean_squared_error(y_test, y_pred))
print('CNN_RMSE:', error)

CNN_RMSE: 0.057947135804246666


## SPLINES

In [37]:
model = Earth(verbose=True)

In [38]:
model.fit(x_train, y_train)

predictions = model.predict(x_test)

error = np.sqrt(mean_squared_error(y_test, predictions, multioutput='uniform_average'))
print('SPLINES_RMSE:', error)

Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.206622  1      0.207  0.000  0.000  
1     0       7    619   0.199500  3      0.200  0.034  0.033  
2     0       4    1325  0.191263  5      0.192  0.074  0.072  
3     0       4    2564  0.184093  7      0.185  0.109  0.106  
4     0       4    160   0.174032  9      0.175  0.158  0.154  
5     0       7    373   0.170895  11     0.172  0.173  0.168  
6     0       7    1079  0.166465  13     0.168  0.194  0.188  
7     0       7    1062  0.157974  15     0.159  0.235  0.229  
8     0       7    1390  0.146847  17     0.148  0.289  0.282  
9     0       7    1454  0.145301  19     0.147  0.297  0.289  
10    0       4    1349  0.139539  21     0.141  0.325  0.316  
11    0       7    16    0.133962  23     0.136  0.352  0.343  
12    0       7  

C:\Users\vasia\anaconda3\envs\myenv\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()


1     71  80     0.14  0.144  0.335  0.301  
2     49  79     0.14  0.144  0.335  0.302  
3     11  78     0.14  0.144  0.335  0.302  
4     50  77     0.14  0.144  0.335  0.302  
5     51  76     0.14  0.144  0.335  0.303  
6     80  75     0.14  0.144  0.335  0.303  
7     56  74     0.14  0.144  0.335  0.304  
8     4   73     0.14  0.144  0.335  0.304  
9     62  72     0.14  0.144  0.335  0.305  
10    68  71     0.14  0.144  0.335  0.305  
11    70  70     0.14  0.144  0.335  0.306  
12    65  69     0.14  0.143  0.335  0.306  
13    76  68     0.14  0.143  0.335  0.306  
14    34  67     0.14  0.143  0.335  0.307  
15    17  66     0.14  0.143  0.335  0.307  
16    48  65     0.14  0.143  0.335  0.308  
17    9   64     0.14  0.143  0.335  0.308  
18    30  63     0.14  0.143  0.335  0.309  
19    31  62     0.14  0.143  0.335  0.309  
20    57  61     0.14  0.143  0.335  0.309  
21    24  60     0.14  0.143  0.335  0.310  
22    54  59     0.14  0.143  0.335  0.310  
23    74  

C:\Users\vasia\anaconda3\envs\myenv\lib\site-packages\pyearth\earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]
